# Simulado para AF de MEGADADOS

A prova tem duração de **2h00**. Veja mais informações no Blackboard!

Não se esqueça de **anexar o notebook** no **Blackboard** ao final da prova. Deixe as células **executadas**!

**NOME**: SEU NOME AQUI!

## Importante

No VSCode ou ao utilizar o Docker (Spark) **abra apenas a pasta da prova**. É proibido abrir a pasta das aulas.

Sugestão: Crie uma pasta e copie o `.env` para lá (assim não precisará da pasta aulas). Crie um **venv** nesta pasta e instale as dependências da prova.

```console
pip install -r requirements.txt
```

Já deixe seu ambiente (pasta com `.env` + **veenv**) preparado para a prova, de tal forma que no dia dia prova baste copiar o conteúdo da prova para a pasta já preparada.

## Spark

Comandos para inicialização do **Spark**. Para macOS e linux, utilize:

```bash
docker run \
    -it \
    --rm \
    -p 8888:8888 \
    -p 4040:4040 \
    -v "`pwd`":/home/jovyan/work \
    jupyter/pyspark-notebook


```

Se estiver no Windows estes comandos, utilize:

- No Powershell: `docker run -it --rm -p 8888:8888 -p 4040:4040 -v ${PWD}:/home/jovyan/work jupyter/pyspark-notebook`

- No Prompt de comando: `docker run -it --rm -p 8888:8888 -p 4040:4040 -v %cd%:/home/jovyan/work jupyter/pyspark-notebook`

Agora abra esse notebook lá no container!

## Insper autograding!

Para receber feedback dos exercício, iremos utilizar o `insper autograding`.

In [1]:
# !pip install -U git+https://github.com/macielcalebe/insperautograding.git

## Como resolver os exercícios?

Crie a base da prova em sua máquina (Exercício 1). Utilize o notebook, MySQL Workbench ou o conector para testar as queries e soluções. Quando estiver bastante certo de que a resposta está correta, faça a submissão para o servidor.

## Base de dados

Em alguns exercícios, iremos utilizar a base de dados `af_md_23_2`.

Execute o script `mensagens.sql` no Workbench para criar e popular a base.

## Import das bibliotecas

Vamos realizar o import das bibliotecas.

In [ ]:
import mysql.connector
import os
import insperautograder.jupyter as ia
from functools import partial
from dotenv import load_dotenv

E vamos criar nosso HELPER de conexão com o banco! Perceba que, uma vez configurado o `.env` não precisaremos mais informar usuários, senhas e URLs!

In [ ]:
load_dotenv(override=True)

def get_connection_helper(database):

    def run_db_query(connection, query, args=None):
        with connection.cursor() as cursor:
            results = cursor.execute(query, args, multi=True)
            for i, result in enumerate(results):
                if result.with_rows:
                    print(f"Resultado query {i}:")
                    for line in result.fetchall():
                        print(line)
                else:
                    print(f"Query {i} executada!")

    connection = mysql.connector.connect(
        host=os.getenv("MD_DB_SERVER"),
        user=os.getenv("MD_DB_USERNAME"),
        password=os.getenv("MD_DB_PASSWORD"),
        database=database,
    )
    return connection, partial(run_db_query, connection)


connection, db = get_connection_helper("af_md_23_2")

In [ ]:
# !pip install pyspark

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("MinhaAplicacao").getOrCreate()
sc = spark.sparkContext
sc

### Notas

As primeiras cinco questões, que possuem correção automática, valem 60% da nota da prova.

Na API de correção automática a nota de cada questão será ponderada pelo seu peso. A nota será apresentada no intervalo 0 a 10, multiplique por 0.6 para saber a nota final considerando toda a prova.

Para conferir a nota da correção automática da prova, utilize:

In [ ]:
ia.grades(task="af_md_23_2")

In [ ]:
ia.grades(by="TASK", task="af_md_23_2")

In [ ]:
ia.grades(by="TASK")

In [ ]:
ia.average(excluded_count=2)

**Exercício 1**:

O arquivo `data/timestamps.txt` contém uma lista de timestamps indicando quando foram feitos acessos a um sistema, um por linha. Cada timestamp é uma string no formato `YYYY-MM-DD HH:MM:SS`. Escreva uma função em Spark que recebe um rdd e um ano (nesta ordem) e retorna quantos acessos houveram em um determinado ano.

In [ ]:
def total_acessos(rdd, ano):
    # Altere esta função!
    return 0

rdd = sc.textFile("data/timestamps.txt")

print(total_acessos(rdd, "2022"))
print(total_acessos(rdd, "2023"))

Após testar localmente e considerar sua solução correta, faça o envio clicando no botão abaixo!

In [ ]:
ia.sender(answer="total_acessos", task="af_md_23_2", question="ex01spark", answer_type="pycode")

**Exercício 2**:

Ainda no arquivo `data/timestamps.txt`, escreva uma função em Spark que recebe um rdd e retorna em ordem decrescente as 20 datas em que houveram mais acessos junto da quantidade de acessos em cada data.

In [ ]:
def top20(rdd):
    # Altere esta função!
    return rdd

top20(rdd)

Após testar localmente e considerar sua solução correta, faça o envio clicando no botão abaixo!

In [ ]:
ia.sender(answer="total_acessos", task="af_md_23_2", question="ex02spark", answer_type="pycode")

**Exercício 3**:

Vimos na primeira parte do curso como podemos utilizar índices para acelerar a performance de *queries*. RDBMS como o MySQL suportam, inclusive, a construção de índices por múltiplas colunas.

Entretanto, em certas situações, como quando são realizadas primariamente buscar por igualdade em múltiplas colunas, uma alternativa é utilizar **hashing** de índices compostos. Nesta técnica, ao invés de construir um índice composto por múltiplas colunas e realizar as consultas neste formato:

```mysql
SELECT t.res1,
       t.res2
FROM tabela t
WHERE t.col1 = 'algum valor'
  AND t.col2 = 'outro valor'
  AND t.col3 = 'criterio de busca'
```

É construida manualmente uma coluna que contém o **hash da concatenação das colunas utilizadas na query**. Então, é criado um índice apenas na coluna que contém o hash. Assim, a *query* de consulta passa a ser, por exemplo:


```mysql
SELECT t.res1,
       t.res2
FROM tabela t
WHERE t.coluna_hash = MD5(CONCAT('algum valor', 'outro valor', 'criterio de busca'))
```

Neste exercício, você deve construir uma *trigger* para atualizar a coluna chamada `coluna_hash` da tabela de `mensagens` quando ocorrer um **update** na tabela de mensagens.

A tabela estará assim:
| id | col1    | col2     | col3    | mensagem                                 |
|----|---------|----------|---------|------------------------------------------|
| 1  | ANA     | SILVA    | ABACATE | Oi, bom dia                              |
| 2  | MARIA   | PEREIRA  | PERA    | Me liga quando puder                     |
| 3  | JOSE    | FERREIRA | ABACATE | Cade vc?                                 |
| 4  | ANTONIO | SILVA    | MACA    | Qual o status do pedido?                 |
| 5  | CAMILA  | WOK      | PERA    | Conseguiu analisar?                      |
| 6  | DANILO  | SCHINLE  | MACA    | Tudo certo pro jantar, vc vem?           |
| 7  | PEDRO   | PACKEL   | BANANA  | Complicado, vai ter que conferir com ela |
| 8  | MANOELA | NANTES   | PERA    | O lugar era lindo                        |
| 9  | CELSO   | ALIVERRI | ABACATE | A visao eh adequada                      |
| 10 | ELLEN   | WETLKS   | UVA     | Vamos juntos                             |

Por exemplo, após criar a coluna `coluna_hash`, ao fazer um *update* no id `1`, ficará assim:

| id | col1    | col2     | col3    | mensagem                                 | coluna_hash                              |
|----|---------|----------|---------|------------------------------------------|------------------------------------------|
| 1  | ANA     | SILVA    | UVA     | Oi, bom dia                              | 2d0b14a194488b9080e080c8a7d82ef88fddbe1e |
| 2  | MARIA   | PEREIRA  | PERA    | Me liga quando puder                     |                                          |
| 3  | JOSE    | FERREIRA | ABACATE | Cade vc?                                 |                                          |
| 4  | ANTONIO | SILVA    | MACA    | Qual o status do pedido?                 |                                          |
| 5  | CAMILA  | WOK      | PERA    | Conseguiu analisar?                      |                                          |
| 6  | DANILO  | SCHINLE  | MACA    | Tudo certo pro jantar, vc vem?           |                                          |
| 7  | PEDRO   | PACKEL   | BANANA  | Complicado, vai ter que conferir com ela |                                          |
| 8  | MANOELA | NANTES   | PERA    | O lugar era lindo                        |                                          |
| 9  | CELSO   | ALIVERRI | ABACATE | A visao eh adequada                      |                                          |
| 10 | ELLEN   | WETLKS   | UVA     | Vamos juntos                             |                                          |

**Obs**:
- Execute o script `mensagens.sql` para construir a base de dados;
- Faça o DDL, em seu servidor local, para criar a coluna `coluna_hash` com array `CHAR` de tamanho 40. No servidor de testes esta coluna já existe e você não deve enviar esta parte;
- Imagine que o índice já existe na coluna de hash (não é relevante para o exercício que ele realmente exista);
- A coluna `coluna_hash` deve ser calculada com o uso da função `SHA1` a partir da concatenação dos colunas `col1`, `col2` e `col3` (nesta ordem e sem espaçadores).


In [ ]:
sql_ex03 = """
-- Sua Trigger AQUI!
"""

db(sql_ex03)

Após testar localmente e considerar sua solução correta, faça o envio clicando no botão abaixo!

In [ ]:
ia.sender(answer="sql_ex03", task="af_md_23_2", question="ex03trigger", answer_type="pyvar")

**Exercício 04**:

Considere que foi criado o usuário `u_consult` com login a partir de qualquer host.

O usuário possui permissão de **LEITURA** e **INSERÇÃO** na tabela de `mensagens` utilizada no exercício anterior. Entretanto, ele não deveria poder **INSERIR** na tabela `mensagens`.

Faça as atualizações, revogando as permissões não mais necessárias.

In [ ]:
sql_ex04 = """
-- Seu SQL AQUI!
"""

db(sql_ex04)

Após testar localmente e considerar sua solução correta, faça o envio clicando no botão abaixo!

In [ ]:
ia.sender(answer="sql_ex04", task="af_md_23_2", question="ex04perm", answer_type="pyvar")

**Exercício 5**:

Um pesquisador irá analisar parte dos dados de um projeto científico para identificação de exoplanetas.

Os dados do projeto são armazenados em um servidor com quatro HDs de 10TB configurados em RAID 5.

Durante o projeto, surgiram questões como:

*"Considerando que são recebidos 14 GB por semana, quantos dias serão necessários para que se atinja 75% da capacidade disponível para armazenamento no servidor?"*

Para responder questões como esta rapidamente, você deve construir uma função `calc_dias_armazenamento` em python que recebe, nesta ordem:

- `tb_por_hd`: quantidade de terabytes de cada um dos quatro HDs;
- `gb_semana`: quantidade de novos dados recebidos a cada semana, em GB;
- `perc`: capacidade percentual limite que se quer analisar. O percentual será passado no intervalo `[0.0 ... 1.0]`.

Como resposta, você deve devolver a quantidade inteira de dias até que que a capacidade percentual seja atingida.

**Obs**:
- Considere o sistema internacional de unidades, ou seja, 1TB = 1000GB e 1GB=1000MB.

In [ ]:
def calc_dias_armazenamento(tb_por_hd, gb_semana, perc):
    # Altere esta função!
    return 0

In [ ]:
# Alguns testes locais
assert calc_dias_armazenamento(10, 14000, 0.5) == 7
assert calc_dias_armazenamento(20, 12800, 0.9) == 29

Após testar localmente e considerar sua solução correta, faça o envio clicando no botão abaixo!

In [ ]:
ia.sender(answer="calc_dias_armazenamento", task="af_md_23_2", question="ex05python", answer_type="pycode")

**Exercício 6**: (**Nota: 1,5**)

Na atividade da próxima *sprint*, foi atribuido ao seu time de analistas de negócio a entrega de um *Dashboard* para acompanhamento dos números de vendas da empresa (vendas por região, vendas por categoria de produto, vendas por canal, etc.)

Dado que isolamento é uma das propriedades fundamentais de bancos de dados relacionais e considerando que parte da especificação envolve:

- O acompanhamento será feito em tempo real;
- É ok ocorrerem imprecisões momentâneas.

Justifique **qual nível de isolamento de transações** você recomendaria que fosse utilizado no projeto.

Esta questão será corrigida considerando a seguinte rubrica:

| Conceito | Nota | Descrição                                                                                                                                               |
|:----------:|----------:|:---------------------------------------------------------------------------------------------------------------------------------------------------------|
| I        | 0.0 |Apenas citou o assunto ou alguns fatos sem explicações                                                                                                  |
| D        | 0.3 |Explicou superficialmente o assunto ou fatos mas sem muitos detalhes conclusivos                                                                        |
| C        | 0.6 |Explicou com detalhes, apresentando definições concretas.                                                                                               |
| B        | 0.8 |Explicou com detalhes, apresentando definições concretas e exemplos de uso.                                                                            |
| A        | 1.0 |Explicou com detalhes, apresentando definições concretas, exemplos de uso e ainda outros tópicos correlatos, fazendo uma conexão lógica entre eles. |

<div class="alert alert-success">

Sua resposta AQUI!

</div>

**Exercício 7**:

Considere um cenário onde projetos são propostos por empresas em busca de financiamento. Cada projeto é proposto por apenas uma empresa, mas a empresa pode propor muitos projetos.

Os projetos podem ou não receber apoio financeiro. Esta decisão é tomada por meio de uma votação, onde usuários membros de um grupo qualquer decidirão se apoiam ou não cada projeto. Cada usuário vota apenas uma vez em cada projeto (`S: sim`, `N: não`, `A: anula`).

Considerando esta tabela de `votos`:

| id_usuario(PK) | nome           | id_projeto(PK) | projeto     | descricao                              | id_empresa | empresa | voto |
|----------------|----------------|----------------|-------------|----------------------------------------|------------|---------|------|
| 20             | Antonio Soares | 1              | Sockerator  | Emparelhador de meias                  | 50         | UniX    | S    |
| 21             | Marina Pereira | 1              | Sockerator  | Emparelhador de meias                  | 50         | UniX    | S    |
| 22             | Camila Mert    | 1              | Sockerator  | Emparelhador de meias                  | 50         | UniX    | N    |
| 20             | Antonio Soares | 2              | Snacktapult | Lança petiscos diretamente para a boca | 51         | Abx     | S    |
| 22             | Camila Mert    | 2              | Snacktapult | Lança petiscos diretamente para a boca | 51         | Abx     | N    |
| 20             | Antonio Soares | 3              | LazyHat     | Um boné com apoio para petiscos        | 50         | UniX    | A    |
| 21             | Marina Pereira | 3              | Lazy Hat    | Um boné com apoio para petiscos        | 50         | UniX    | A    |

Chamamos de normalização o processo de rearranjar o banco de dados para que obedeça às várias formas normais. Assim, reduzimos a redundância, aumentamos o desempenho e a integridade de dados.

**a)** (**Nota: 1,0**) Ao analisar a tabela, um aluno fez a seguinte afirmação:

*"A tabela de `votos` está na `2NF` mas não está na `1NF`"*

A afirmação do aluno está correta? Justifique detalhadamente.

Esta questão será corrigida considerando a seguinte rubrica:

| Conceito | Nota | Descrição                                                                                                                                               |
|:----------:|----------:|:---------------------------------------------------------------------------------------------------------------------------------------------------------|
| I        | 0.0 |Apenas citou o assunto ou alguns fatos sem explicações                                                                                                  |
| D        | 0.3 |Explicou superficialmente o assunto ou fatos mas sem muitos detalhes conclusivos                                                                        |
| C        | 0.6 |Explicou com detalhes, apresentando definições concretas.                                                                                               |
| B        | 0.8 |Explicou com detalhes, apresentando definições concretas e exemplos de uso.                                                                            |
| A        | 1.0 |Explicou com detalhes, apresentando definições concretas, exemplos de uso e ainda outros tópicos correlatos, fazendo uma conexão lógica entre eles. |

<div class="alert alert-success">

Sua resposta AQUI!

</div>

**b**) (**Nota: 1,5**) Baseado na tabela denormalizada fornecida, normalize os dados até a 3ª Forma Normal (`3NF`), garantindo:

1. Eliminação de redundâncias.
2. Integridade referencial entre as entidades.
3. Relacionamento correto entre jogos, times, jogadores e técnicos.

Crie no **MySQL Workbench** ou **Draw.io** o **Diagrama do modelo relacional (EER Diagram)** que representa a base de dados normalizada.

**Instruções:**
- Salve o DER na pasta `resposta_der`. Utilize formato **png** ou **jpg**.
- Insira o caminho do arquivo no notebook usando uma tag para exibir a imagem.

**Critérios de Avaliação (Rubrica):**

| Conceito | Nota Porcentual | Descrição                                                                                                                                               |
|:----------:|----------:|:---------------------------------------------------------------------------------------------------------------------------------------------------------|
| I        | 0.0 |Insuficiente, as modificações propostas não consideram as informações a serem armazenadas ou consideram um cenário diferente do proposto |
| D        | 0.3 |Não está na 1NF, mas a solução está em desenvolvimento e permite armazenar boa parte das informações |
| C        | 0.6 |Está na 1NF, sem erros, redundâncias ou ineficiências | |
| B        | 0.8 |Está na 2NF, sem erros, redundâncias ou ineficiências | |
| A        | 1.0 |Está na 3NF, sem erros, redundâncias ou ineficiências |

<div class="alert alert-success">

Seu diagrama do workbench AQUI!

<img src="resposta_der/exemplo.png">

</div>

### Conferindo as notas!

In [ ]:
ia.grades(task="af_md_23_2")

In [ ]:
ia.grades(by="TASK", task="af_md_23_2")

In [ ]:
ia.grades(by="TASK")

In [ ]:
ia.average(excluded_count=2)

## Entrega!

É hora de entregar. Envie um **ZIP** (não **RAR**) e finalize o teste no proctorio!